![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/arc/images/microsoft-small-logo.png)
 
## Deploy a PostgreSQL server group on an existing Azure Arc data cluster
 
This notebook walks through the process of deploying a PostgreSQL server group on an existing Azure Arc data cluster.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* Make sure you have the target Azure Arc data cluster already created.

<span style="color:red"><font size="3">Please press the "Run All" button to run the notebook</font></span>

### **Check prerequisites**

In [ ]:
import sys,os,json,subprocess
def run_command():
    print("Executing: " + cmd)
    output = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True )
    if output.returncode != 0:
        print(f'Command: {cmd} failed \n')
        print(f'\t>>>Error output: {output.stderr.decode("utf-8")}\n')
        sys.exit(f'exit code: {output.returncode}\n')
    print(f'Successfully executed: {cmd}')
    print(f'\t>>>Output: {output.stdout.decode("utf-8")}\n')
    return output.stdout.decode("utf-8")
cmd = 'azdata --version'
out = run_command()


#### **Ensure Postgres Server Group name and password exist**

In [ ]:
# Required Values
env_var =  "AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_NAME" in os.environ
if env_var:
    server_group_name = os.environ["AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_NAME"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_NAME was not defined. Exiting\n')
env_var = "AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_PASSWORD" in os.environ
if env_var:
    postgres_password = os.environ["AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_PASSWORD"]
else:
    sys.exit(f'environment variable: AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_PASSWORD was not defined. Exiting\n')   


#### **Get optional parameters for the PostgreSQL server group**

In [ ]:
server_group_workers = os.environ["AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_WORKERS"]
server_group_port = os.environ.get("AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_PORT")
server_group_cores_request = os.environ.get("AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_CORES_REQUEST")
server_group_cores_limit = os.environ.get("AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_CORES_LIMIT")
server_group_memory_request = os.environ.get("AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_MEMORY_REQUEST")
server_group_memory_limit = os.environ.get("AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_MEMORY_LIMIT")

### **Installing PostgreSQL server group**

In [ ]:
print (f'Creating a PostgreSQL server group on Azure Arc')

workers_option = f' -w {server_group_workers}' if server_group_workers else ""
port_option = f' --port "{server_group_port}"' if server_group_port else ""
cores_request_option = f' -cr "{server_group_cores_request}"' if server_group_cores_request else ""
cores_limit_option = f' -cl "{server_group_cores_limit}"' if server_group_cores_limit else ""
memory_request_option = f' -mr "{server_group_memory_request}Mi"' if server_group_memory_request else ""
memory_limit_option = f' -ml "{server_group_memory_limit}Mi"' if server_group_memory_limit else ""

os.environ["AZDATA_PASSWORD"] = postgres_password
cmd = f'azdata arc postgres server create -n {server_group_name}{workers_option}{port_option}{cores_request_option}{cores_limit_option}{memory_request_option}{memory_limit_option}'
out=run_command()